# LINFO2263 Assignement 2 
by Sarra Laksaci


---




In [ ]:
#IMPORTS 
import nltk
import math
import numpy as np 
import pandas as pd
from nltk import ngrams
from nltk import bigrams
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Load corpus with PlainTextReader from nltk.corpus.readers 
input_directory="." #normalement corpora/
corpus = nltk.corpus.reader.PlaintextCorpusReader(input_directory,'.*.txt')

In [ ]:
paragraphs= corpus.paras()
print(len(paragraphs))

1505


In [ ]:
"""text=open("/content/p2_sherlock.txt")
text_plain=text.read()
text_low=text_plain.lower()"""

In [ ]:
delete= ["\"", "'", ",", ".", ":", "?", "!", ".\"", "?\"", "!\"", "-", ",\"", "--"]

In [ ]:
"""text_w_out_punc= "".join([char for char in text_low if char not in delete ])"""

In [ ]:
"""new_file= open("/content/new_corpus.txt",mode="w+")
new_file.write(text_w_out_punc)
new_file.close()"""

In [ ]:
"""new_corpus= nltk.corpus.reader.plaintext.PlaintextCorpusReader(".","new_corpus.txt")"""

In [ ]:
#Form vocabulary 
vocabulary=[]
for word in corpus.words():
  if (word not in vocabulary):
    if word not in delete:
      word=word.lower()
      vocabulary.append(word)

In [ ]:
len(vocabulary)

12367

In [ ]:
words=[]
for word in corpus.words():
  if word not in delete:
    words.append(word.lower())

In [ ]:
fifgrams = ngrams(words, 5)
i=0

for grams in fifgrams:
  print (grams)


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
('the', 'thin', 'peninsula', 'of', 'firm')
('thin', 'peninsula', 'of', 'firm', 'peaty')
('peninsula', 'of', 'firm', 'peaty', 'soil')
('of', 'firm', 'peaty', 'soil', 'which')
('firm', 'peaty', 'soil', 'which', 'tapered')
('peaty', 'soil', 'which', 'tapered', 'out')
('soil', 'which', 'tapered', 'out', 'into')
('which', 'tapered', 'out', 'into', 'the')
('tapered', 'out', 'into', 'the', 'widespread')
('out', 'into', 'the', 'widespread', 'bog')
('into', 'the', 'widespread', 'bog', 'from')
('the', 'widespread', 'bog', 'from', 'the')
('widespread', 'bog', 'from', 'the', 'end')
('bog', 'from', 'the', 'end', 'of')
('from', 'the', 'end', 'of', 'it')
('the', 'end', 'of', 'it', 'a')
('end', 'of', 'it', 'a', 'small')
('of', 'it', 'a', 'small', 'wand')
('it', 'a', 'small', 'wand', 'planted')
('a', 'small', 'wand', 'planted', 'here')
('small', 'wand', 'planted', 'here', 'and')
('wand', 'planted', 'here', 'and', 'there')
('p

In [ ]:
matrix={}
cpt=0
n = 5
fifgrams = ngrams(words, n)
for gram in fifgrams:
  key_list=[]
  if(cpt==0):
    #Creat context window of 1st word ! aka pred1=None pred2=None succ1=2nd word and succ2=3rd word
    key_list.append((gram[0],gram[1]))
    key_list.append((gram[0],gram[2]))
   
    #Creat context window of 2st word ! aka pred1=None pred2=1st word succ1=3nd word and succ2=4rd word
    second=[]
    key_list.append((gram[1],gram[0]))
    key_list.append((gram[1],gram[2]))
    key_list.append((gram[1],gram[3]))
   

  key_list.append((gram[2],gram[0]))
  key_list.append((gram[2],gram[1]))
  key_list.append((gram[2],gram[3]))
  key_list.append((gram[2],gram[4]))

  for key in key_list:
    if key in matrix.keys():
      matrix[key]+=1
    else:
      matrix[key]=1
  cpt+=1


In [ ]:
#Creat context window of last word
key_list.append((gram[4],gram[3]))
key_list.append((gram[4],gram[2]))
#Creat context window of before last word
key_list.append((gram[3],gram[1]))
key_list.append((gram[3],gram[2]))
key_list.append((gram[3],gram[4]))
for key in key_list:
  if key in matrix.keys():
    matrix[key]+=1
  else:
    matrix[key]=1

In [ ]:
epsilon=0.0001
bottom = sum(matrix.values())+ epsilon * len(vocabulary) * len(vocabulary)

In [ ]:
def proba_combination(w,v,epsilon,bottom):
  M=0
  key = (w,v)
  if ( key in matrix.keys()):
    M=matrix[(w,v)]
  
  return((M+epsilon)/bottom)
  

In [ ]:
proba_combination("the","of",epsilon,bottom)

0.0038829534508515854

In [ ]:
def proba_target(w):
  p=0
  for key,value in matrix.items():
    if (key[0]==w):
      p+=proba_combination(key[0],key[1],epsilon,bottom)
  return(p)

In [ ]:
def proba_context(v):
  p=0
  for key,value in matrix.items():
   # print( f" \n {key}  \n  {key[0]} , {key[1]} \n {key[1]} = {v}")
    if (key[1]==v):

      p+=proba_combination(key[0],key[1],epsilon,bottom)
  return p

In [ ]:
def ppmi(w,v):

  if(w==v):
    return 0 
  p1= proba_target(w)
  p2= proba_context(v)
  solo_probas= p1 * p2
  div=proba_combination(w,v,epsilon,bottom)/solo_probas
  pmi= math.log2(  div)
  return (max([pmi,0]))

In [ ]:
def top_five_ppmi(target):
  all=[ppmi(target,v) for v in vocabulary]
  all =np.array(all)
  idx_best=all.argsort()[-6:][::-1]
  return ([vocabulary[i] for i in idx_best])

In [ ]:
top_five_ppmi("glimpse")

['catching', 'catch', 'caught', 'distant', 'get', 'get']

In [ ]:
top_five_ppmi("school")
#always gives 0 or 53 for context
#target proba is re calculated for each context .. can be optimized witg DP

['established', 'private', 'private', 'read', 'read', 'three']

In [ ]:
top_five_ppmi("incident")

['instructive', 'foolish', 'clearing', 'grotesque', 'record', 'appeared']

In [ ]:
top_five_ppmi("white")

['swirl', 'wool', 'dabbled', 'froth', 'fence', 'wooden']

In [ ]:
top_five_ppmi("sherlock")

['incredulously', 'holm', '://', 'http', 'es', 'contents']